In [14]:
from pathlib import Path
import os 
import sys
import numpy as np
import json

root = Path().resolve().parent

root_str = str(root)
if root_str not in sys.path:
    sys.path.insert(0, root_str)

if Path.cwd() != root:
    os.chdir(root)
    
from importlib import reload
import src.evaluate

### Rotowire

In [2]:
from jsonschema import Draft7Validator
from typing import Any, Dict, List, Tuple

def _ok(value: Any, allowed_types) -> bool:
    """Helper function to check key-wise type."""
    if not isinstance(allowed_types, list):
        allowed_types = [allowed_types]
    return (
        (value is None and "null" in allowed_types) or
        (isinstance(value, int)  and "integer" in allowed_types) or
        (isinstance(value, str)  and "string"  in allowed_types)
    )

def _score_section(data: List[Dict[str, Any]] | None,
                   definition: Dict[str, Any],
                   min_items: int | None,
                   max_items: int | None) -> Tuple[int, int]:
    """
    Returns (valid, total) for the section.
    If data is None (missing) or length violates min/max, returns (0, 1) → 0 %.
    """
    if data is None:
        return 0, 1                              # missing section → 0 %
    if (min_items and len(data) < min_items) or \
       (max_items and len(data) > max_items):
        return 0, 1                              # length violation → 0 %

    props     = definition["properties"]
    required  = set(definition.get("required", []))
    allow_add = definition.get("additionalProperties", False)

    total = valid = 0
    for obj in data:
        # Required keys first
        for key in required:
            total += 1
            if key in obj and _ok(obj[key], props[key]["type"]):
                valid += 1
                
        # Then looking at the other ones
        for key, val in obj.items():
            if key in required:
                continue
            total += 1
            if key in props:
                if _ok(val, props[key]["type"]):
                    valid += 1
            elif allow_add:
                valid += 1                        # Allowed extras
            # else: extra - invalid (counts as 0)

    return valid, total if total else (0,1)       # avoid div-by-zero

def compliance_breakdown(schema: Dict[str, Any],
                         record: Dict[str, Any]) -> Dict[str, float]:
    """Percentage compliance + real Draft-07 errors, handles all edge-cases."""
    validator = Draft7Validator(schema)
    errors    = list(validator.iter_errors(record))

    # Pull defs & array length rules from the schema
    team_def   = schema["definitions"]["team"]
    player_def = schema["definitions"]["player"]

    teams_schema   = schema["items"]["properties"]["teams"]
    players_schema = schema["items"]["properties"]["players"]

    t_valid, t_total = _score_section(
        record.get("teams"),
        team_def,
        teams_schema.get("minItems"),
        teams_schema.get("maxItems")
    )
    p_valid, p_total = _score_section(
        record.get("players"),
        player_def,
        players_schema.get("minItems"),
        players_schema.get("maxItems")
    )

    team_compliance = t_valid / t_total
    player_compliance = p_valid / p_total

    return {
        "team_compliance_%":   team_compliance * 100,
        "player_compliance_%": player_compliance * 100,
        "overall_compliance_%": float(np.mean([team_compliance, player_compliance])) * 100,
        "errors": errors
    }


In [3]:
path = "data/clean/rotowire/"

# Example usage:
with open(path + "schema.json") as sf, open(path + "clean.json") as rf:
    schema = json.load(sf)
    record = json.load(rf)[0]["output"]

result = compliance_breakdown(schema, record)
print(f"Teams:  {result['team_compliance_%']:.1f}%")
print(f"Players:{result['player_compliance_%']:.1f}%")
print(f"Overall:{result['overall_compliance_%']:.1f}%")
for err in result["errors"]:
    print(f"– {err.message} (at {list(err.absolute_path)})")
print(result)

Teams:  100.0%
Players:100.0%
Overall:100.0%
{'team_compliance_%': 100.0, 'player_compliance_%': 100.0, 'overall_compliance_%': 100.0, 'errors': []}


In [4]:
record

{'teams': [{'team': 'Suns', 'losses': 38, 'total_points': 87, 'wins': 39},
  {'team': 'Jazz', 'losses': 42, 'total_points': 85, 'wins': 34}],
 'players': [{'player': 'Gordon Hayward',
   'assists': None,
   'blocks': None,
   '3_pointers_made': None,
   'field_goals_attempted': 18,
   'minutes_played': None,
   'points': 21,
   'total_rebounds': None,
   'steals': None},
  {'player': 'Rudy Gobert',
   'assists': 2,
   'blocks': 1,
   '3_pointers_made': None,
   'field_goals_attempted': None,
   'minutes_played': 38,
   'points': 8,
   'total_rebounds': 15,
   'steals': 1},
  {'player': 'Rodney Hood',
   'assists': None,
   'blocks': None,
   '3_pointers_made': None,
   'field_goals_attempted': None,
   'minutes_played': None,
   'points': 17,
   'total_rebounds': None,
   'steals': None},
  {'player': 'Brandan Wright',
   'assists': 1,
   'blocks': 4,
   '3_pointers_made': None,
   'field_goals_attempted': None,
   'minutes_played': 33,
   'points': 14,
   'total_rebounds': 3,
   'stea

In [21]:
reload(src.evaluate)
from src.evaluate import RotowireEvaluator

eval = RotowireEvaluator(schema)

eval.compliance_breakdown(json.dumps(record))

{'is_valid': 1,
 'team_compliance_%': 1.0,
 'player_compliance_%': 1.0,
 'overall_compliance_%': 1.0,
 'errors': []}

In [22]:
eval.compliance_breakdown(str(record))

{'is_valid': 0,
 'team_compliance_%': 1.0,
 'player_compliance_%': 0.46296296296296297,
 'overall_compliance_%': 0.7314814814814815,
 'errors': []}

In [23]:
record_2 = """{'teams': [{'team': 'Suns', 'losses': 38, 'total_points': 87, 'wins': 39},
  {'team': 'Jazz', 'losses': 42, 'total_points': 85, 'wins': 34}],
 'players': [{'player': 'Gordon Hayward',
   'assists': null,
   'blocks': null,
   '3_pointers_made': null,
   'field_goals_attempted': 18,
   'minutes_played': null,
   'points': 21,
   'total_rebounds': null,
   'steals': null},
  {'player': 'Rudy Gobert',
   'assists': 2,
   'blocks': 1,
   '3_pointers_made': null,
   'field_goals_attempted': null,
   'minutes_played': 38,
   'points': 8,
   'total_rebounds': 15,
   'steals': 1},
  {'player': 'Rodney Hood',
   'assists': null,
   'blocks': null,
   '3_pointers_made': null,
   'field_goals_attempted': null,
   'minutes_played': null,
   'points': 17,
   'total_rebounds': null,
   'steals': null},
  {'player': 'Brandan Wright',
   'assists': 1,
   'blocks': 4,
   '3_pointers_made': null,
   'field_goals_attempted': null,
   'minutes_played': 33,
   'points': 14,
   'total_rebounds': 3,
   'steals': 3},
  {'player': 15,
   'assists': null,
   'blocks': null,
   '3_pointers_made': null,
   'field_goals_attempted': null,
   'minutes_played': null,
   'points': 4,
   'total_rebounds': null,
   'steals': null},
  {'player': 'Gerald Green',
   'assists': null,
   'blocks': null,
   '3_pointers_made': 3,
   'field_goals_attempted': null,
   'minutes_played': 22,
   'points': 24,
   'total_rebounds': null,
   'ste;als': null}}"""
   
eval.compliance_breakdown(record_2)

{'is_valid': 0,
 'team_compliance_%': 1.0,
 'player_compliance_%': 0.9629629629629629,
 'overall_compliance_%': 0.9814814814814814,
 'errors': []}

In [24]:
eval.evaluate_game(record, record)

{'ident_f1': 1.0,
 'object_attr_F1': 1.0,
 'object_value_acc': 1.0,
 'object_attribute_score': 1.0,
 'overall_score': 1.0}

In [11]:
record

{'teams': [{'team': 'Suns', 'losses': 38, 'total_points': 87, 'wins': 39},
  {'team': 'Jazz', 'losses': 42, 'total_points': 85, 'wins': 34}],
 'players': [{'player': 'Gordon Hayward',
   'assists': None,
   'blocks': None,
   '3_pointers_made': None,
   'field_goals_attempted': 18,
   'minutes_played': None,
   'points': 21,
   'total_rebounds': None,
   'steals': None},
  {'player': 'Rudy Gobert',
   'assists': 2,
   'blocks': 1,
   '3_pointers_made': None,
   'field_goals_attempted': None,
   'minutes_played': 38,
   'points': 8,
   'total_rebounds': 15,
   'steals': 1},
  {'player': 'Rodney Hood',
   'assists': None,
   'blocks': None,
   '3_pointers_made': None,
   'field_goals_attempted': None,
   'minutes_played': None,
   'points': 17,
   'total_rebounds': None,
   'steals': None},
  {'player': 'Brandan Wright',
   'assists': 1,
   'blocks': 4,
   '3_pointers_made': None,
   'field_goals_attempted': None,
   'minutes_played': 33,
   'points': 14,
   'total_rebounds': 3,
   'stea